# Kullanıcı Tabanlı İş Birlikçi Filtreleme ( User-Based Collaborative Filtering )

In [1]:
# Kullanıcıların birbirlerine benzerlikleri üzerinden öneriler yapılır. 
# Örnek olarak aksiyon filmleri izleyen bir kullanıcıya, 
# kendi izleme alışkanlıklarına en benzer izleme geçmişine sahip başka bir kullanıcının izlediği filmlerin önerilmesini verebiliriz.

Bu metod item based collaborative filtering'e benzemektedir. Item based collaborative filtering'de; bir filmin benzer beğenilme davranışlarına sahip diğer filmler ile olan durumlarını değerlendirdik. Bir film ile en çok beğenilme pattern'ına (örüntüsüne) sahip olan filmleri önermiştik.


Lakin item based collaborative filtering'den farkı ise kullanıcılara daha çok odaklanılmasıdır. Örnek olarak A kişisinin hangi filmleri izlediği bulunmaktadır. Daha sonrasında A kişisinin izlediği filmlere benzer film izleyen kişiler bulunmak bulunmaktadır. Bunun yanı sıra A kişisi ile benzer filmleri izleyen kişiler içerisinden de "A kişisi ile en benzer beğenme veya beğenmeme davranışı gösteren kişiler" tespit edileccektir.

Yani bu A kişisi bir film beğenmiş ise diğerlerinin de beğendiği, bu A kişisi bir film beğenmemiş ise diğerlerinin de beğenmemiş olduğu kişiler tespit edilecektir.

In [2]:
# User-Based Collaborative Filtering

# Adım 1: Veri Setinin Hazırlanması
# Adım 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi
# Adım 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek
# Adım 4: Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi
# Adım 5: Weighted Average Recommendation Score'un Hesaplanması
# Adım 6: Çalışmanın Fonksiyonlaştırılması

### Adım 1: Veri Setinin Hazırlanması

In [3]:
import pandas as pd

pd.set_option('display.width', 500)

In [4]:
movie = pd.read_csv('datasets/movie_lens_dataset/movie.csv')
rating = pd.read_csv('datasets/movie_lens_dataset/rating.csv')

In [5]:
df = movie.merge(rating, how="left", on="movieId")
df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
20000792,131254,Kein Bund für's Leben (2007),Comedy,79570.0,4.0,2015-03-30 19:32:59
20000793,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570.0,4.0,2015-03-30 19:48:08
20000794,131258,The Pirates (2014),Adventure,28906.0,2.5,2015-03-30 19:56:32
20000795,131260,Rentun Ruusu (2001),(no genres listed),65409.0,3.0,2015-03-30 19:57:46


In [6]:
comment_counts = pd.DataFrame(df["title"].value_counts())
comment_counts

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [7]:
rare_movies = comment_counts[comment_counts["title"] <= 1000].index

In [8]:
common_movies = df[~df["title"].isin(rare_movies)]
common_movies

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
19985698,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,28195.0,4.0,2014-09-22 20:52:18
19985699,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,51334.0,3.0,2014-09-23 15:53:39
19985700,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,120575.0,2.5,2014-10-08 14:23:39
19985701,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,124998.0,2.5,2014-09-20 22:16:14


In [9]:
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Bu aşamada bir kişiyi incelemek için bir kullanıcı seçmem gerekecek;
# Bu kullanıcıyı "user_movie_df" tablosundan seçeceğiz.
# Bu tabloda ise kullanıcılar, indeks'te saklanmaktadır.
# Bu sebeple indeksten bir seçim işlemi yapacağız.

user_movie_df.index

Float64Index([     1.0,      2.0,      3.0,      4.0,      5.0,      6.0,      7.0,      8.0,      9.0,     10.0,
              ...
              138484.0, 138485.0, 138486.0, 138487.0, 138488.0, 138489.0, 138490.0, 138491.0, 138492.0, 138493.0], dtype='float64', name='userId', length=138493)

In [11]:
# Yukarıda indeksler gözükmektedir.
# Bir seçim yapabilmek için yukarıdaki listeyi bir pandas series'i ne dönüştürüyoruz;

pd.Series(user_movie_df.index)

0              1.0
1              2.0
2              3.0
3              4.0
4              5.0
            ...   
138488    138489.0
138489    138490.0
138490    138491.0
138491    138492.0
138492    138493.0
Name: userId, Length: 138493, dtype: float64

In [12]:
# Şimdi bu pandas series'inden bir seçim yapabilmek için ".sample()" metodunu kullanıyoruz.
# Metoda detaylı bakmak için "https://pandas.pydata.org/docs/reference/api/pandas.Series.sample.html" sitesine gidebilirsiniz.
# içerisine argüman olarak; 1 giriyorum (bir kullanıcıyı seçmesi için)

# DİKKAT !!! Siz de bu kodu çalıştırırken bu notebook'daki sonuçların aynısını alabilmeniz için "random_state=" argümanı 45 girilmiştir.
# Yani örneklem olarak alacağı 1 kişi belirli bir kişidir.

pd.Series(user_movie_df.index).sample(1, random_state=45)

# Böyle bir seçimde bize değer getirmedi. Bize bunun sayısal olarak değeri gerek.

28940    28941.0
Name: userId, dtype: float64

In [13]:
# Bu değeri edinebilmemiz için;

pd.Series(user_movie_df.index).sample(1, random_state=45).values

array([28941.])

In [14]:
# Yukarıdaki kod ile de bir array döndü;
# Bizim bunu sayı olarak döndürebilmemizin yolu integer haline çevirmek;

random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)

# Bu değeri bir değişkene atayacağız.

random_user

28941

Kullanıcı id'imizi bulduk, yukarıda ifade edildiği üzere; eğer ki "random_state=45" argümanı kalıdırlırsa, gerçekten de random sonuçlara göre user belirlenir.

Artık amacımız yukarıda belirlenen kullanıcı hangi filmleri izlemiş bunu bulmaktır. Daha sonrasında bu filmleri ile aynı filmleri izlemiş kullanıcıları bulacağız. Bu kullanıcılar bulunduktan sonra "28941" user ile benzer beğenme davranışı sergileyen kullanıcıları bulup, bunun üzerinden verilen puanlara yönelik bir skor oluşturacağız ve bu skora göre "28941" user'a hangi filmleri önermemiz gerektiğini bulacağız.

### Adım 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi

In [15]:
random_user_df = user_movie_df[user_movie_df.index == random_user]
random_user_df

# Veri setimizi "28941" kullanıcısına indirgemiş olduk.
# Dikkat sütunlarda bütün filmler bulunmaktadır.
# Sütunlarda daha bir indirgeme yapılmamıştır.

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
28941.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Amacımız burada "28941" kullanıcı no'lu user'in izlediği filmleri bulacağız.

random_user_df.dropna(axis=1)

# "28941" nolu kullanıcı 33 film puanlamış.

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),...,Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,
28941.0,3.0,2.0,2.0,3.0,3.0,5.0,4.0,5.0,3.0,3.0,...,3.0,5.0,5.0,5.0,3.0,5.0,5.0,1.0,1.0,1.0


In [17]:
# Şimdi bizim bu filmleri seçebilmemiz için bunların sütun isimlerine erişebiliyor olamız gerekiyor,

random_user_df.dropna(axis=1).columns

Index(['Ace Ventura: Pet Detective (1994)', 'Ace Ventura: When Nature Calls (1995)', 'Aladdin (1992)', 'American President, The (1995)', 'Apollo 13 (1995)', 'Babe (1995)', 'Bullets Over Broadway (1994)', 'Clueless (1995)', 'Disclosure (1994)', 'Forrest Gump (1994)', 'Four Weddings and a Funeral (1994)', 'Home Alone (1990)', 'Jurassic Park (1993)', 'Like Water for Chocolate (Como agua para chocolate) (1992)', 'Little Women (1994)', 'Mr. Holland's Opus (1995)', 'Mrs. Doubtfire (1993)',
       'Much Ado About Nothing (1993)', 'Muriel's Wedding (1994)', 'Nine Months (1995)', 'Operation Dumbo Drop (1995)', 'Piano, The (1993)', 'Postman, The (Postino, Il) (1994)', 'Ready to Wear (Pret-A-Porter) (1994)', 'Remains of the Day, The (1993)', 'Sabrina (1995)', 'Schindler's List (1993)', 'Secret Garden, The (1993)', 'Sense and Sensibility (1995)', 'Shadowlands (1993)', 'Silence of the Lambs, The (1991)', 'Star Trek: Generations (1994)', 'Stargate (1994)'],
      dtype='object', name='title')

In [18]:
# Yukarıdaki çıktıyı ise python listesine çeviriyoruz;
# ve bir değişkene atıyoruz.

movies_watched = random_user_df.dropna(axis=1).columns.tolist()
movies_watched

['Ace Ventura: Pet Detective (1994)',
 'Ace Ventura: When Nature Calls (1995)',
 'Aladdin (1992)',
 'American President, The (1995)',
 'Apollo 13 (1995)',
 'Babe (1995)',
 'Bullets Over Broadway (1994)',
 'Clueless (1995)',
 'Disclosure (1994)',
 'Forrest Gump (1994)',
 'Four Weddings and a Funeral (1994)',
 'Home Alone (1990)',
 'Jurassic Park (1993)',
 'Like Water for Chocolate (Como agua para chocolate) (1992)',
 'Little Women (1994)',
 "Mr. Holland's Opus (1995)",
 'Mrs. Doubtfire (1993)',
 'Much Ado About Nothing (1993)',
 "Muriel's Wedding (1994)",
 'Nine Months (1995)',
 'Operation Dumbo Drop (1995)',
 'Piano, The (1993)',
 'Postman, The (Postino, Il) (1994)',
 'Ready to Wear (Pret-A-Porter) (1994)',
 'Remains of the Day, The (1993)',
 'Sabrina (1995)',
 "Schindler's List (1993)",
 'Secret Garden, The (1993)',
 'Sense and Sensibility (1995)',
 'Shadowlands (1993)',
 'Silence of the Lambs, The (1991)',
 'Star Trek: Generations (1994)',
 'Stargate (1994)']

In [19]:
# Ufak bir kontrol yapmak istersek;

user_movie_df.loc[user_movie_df.index == random_user, user_movie_df.columns == "Schindler's List (1993)"]

# "28941" kullanıcı no'lu user "Schindler's List (1993)" filmini izlemiş ve 5 puan vermiştir.

title,Schindler's List (1993)
userId,
28941.0,5.0


### Adım 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek

In [20]:
# Bu aşamada "28941" kullanıcı numarası ile aynı fimleri izleyen kişilerin ID'lerini bulacağız;

# Bu sebeple veri setimizi 28941 kullanıcı numarasının izlediği filmler ile bir indirgeyelim;

movies_watched_df = user_movie_df[movies_watched]
movies_watched_df

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),...,Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0
4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN
5.0,NaN,NaN,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,5.0,3.0,NaN,3.0,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
138490.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Artık "28941" kullanıcı numarasına sahip kişi ile aynı filmleri veya filmi izleyen kişileri bulacağız.

Lakin buradaki sorunumuz şu ki; "28941" kullanıcı numarasına sahip kişi ile en az bir tane aynı filmi izleyen kişiler bizim veri setimize girmeli mi?

In [21]:
# Bu kararı geliştirici vermelidir.
# Lakin mantıklı düşünce şudur ki belirli bir sayının üzerinde ortak film izleyen kullanıcıları bulmamız gerekir.
# Bunun için de bir eşik değer belirlenecektir.

# Yani eşik değer üzerindeki kullanıcılar çalışmaya dahil olacaktır.

In [22]:
# Şimdi bu hesabı yapabilmek için kullanıcıların kaçar film izlediklerini bulacağız;

user_movie_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Kullanıcıların kaçar film izlediklerini bulabilmek için null olmayan değerleri (notnull) saydırmamız gerekir.

movies_watched_df.notnull().head()

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),...,Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,True
4.0,False,True,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,True,False
5.0,False,False,True,True,True,False,False,False,False,False,...,False,True,False,False,True,True,False,True,False,True


In [24]:
movies_watched_df.notnull().sum().head()

# Aşağıda bakıldığında filmlerin kaç kere izlendiği sayıldı.
# DİKKAT !!! Kullanıcıların kaç film izlediği değil, filmlerin kaç kere izlendiği sayıldı.

# Yani bizim satırlarımızı sütun haline getirmemiz gerekir.

title
Ace Ventura: Pet Detective (1994)        38226
Ace Ventura: When Nature Calls (1995)    20938
Aladdin (1992)                           41844
American President, The (1995)           18162
Apollo 13 (1995)                         47777
dtype: int64

In [25]:
# Satırları sütun haline getirebilmek için ise Transpose alırız.

movies_watched_df.T.head()

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,138484.0,138485.0,138486.0,138487.0,138488.0,138489.0,138490.0,138491.0,138492.0,138493.0
title,,,,,,,,,,,,,,,,,,,,,
Ace Ventura: Pet Detective (1994),NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,3.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0
Ace Ventura: When Nature Calls (1995),NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.0,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
Aladdin (1992),NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,NaN,...,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.5
"American President, The (1995)",NaN,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,4.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apollo 13 (1995),NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.5


In [26]:
# Şimdi yukarıdaki işlemleri transpose alınmış df için uygulayacağız ve bunu bir değişkene atayacağız;

user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count

userId
1.0          1
2.0          2
3.0          4
4.0          6
5.0         11
            ..
138489.0     1
138490.0     7
138491.0     0
138492.0     2
138493.0     9
Length: 138493, dtype: int64

In [27]:
# Üzerinde işlem yapacağımız için yukarıda indeks olan "userID" değişkenini indeks olmaktan kurtaracağız.

user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count

,userId,movie_count
0,1.0,1
1,2.0,2
2,3.0,4
3,4.0,6
4,5.0,11
...,...,...
138488,138489.0,1
138489,138490.0,7
138490,138491.0,0
138491,138492.0,2


In [28]:
user_movie_count[user_movie_count["movie_count"]>20].sort_values("movie_count", ascending = False)

# Aşağıdaki kullanıcılar, "28941" kullanıcı numaralı kişinin izlediği filmlerin en az 21 tanesini izleyen kişilerdir.

,userId,movie_count
94230,94231.0,33
100398,100399.0,33
118204,118205.0,33
15918,15919.0,33
124051,124052.0,33
...,...,...
79214,79215.0,21
79174,79175.0,21
9105,9106.0,21
78515,78516.0,21


In [29]:
# "28941" kullanıcı numaralı kişinin izlediği bütün filmleri izleyen kişi sayısına bakmak istersek;

user_movie_count[user_movie_count["movie_count"] == 33].count()

# Burası karar noktasıdır.
# Bu 17 kişinin yeterli olduğunu düşünüyorsak analizimize bu 17 kişi üzerinden devam etmemiz gerekecektir.

# Lakin biz "28941" kullanıcı numaralı kişinin izlediği filmlerin yarısı kadar film de ortak nokta için yeterlidir diyebiliriz.
# Veya biz "28941" kullanıcı numaralı kişinin izlediği filmlerin 20'si kadarını izleyen kişiler üzerinden analiz yapacağız da diyebiliriz.

userId         17
movie_count    17
dtype: int64

In [30]:
users_same_movies = user_movie_count[user_movie_count["movie_count"]>20]["userId"]
users_same_movies

# Yukarıda """ user_movie_count[user_movie_count["movie_count"]>20] """ df üzerinden "userID"leri seçtik.

129          130.0
155          156.0
157          158.0
183          184.0
294          295.0
            ...   
138207    138208.0
138278    138279.0
138381    138382.0
138414    138415.0
138482    138483.0
Name: userId, Length: 3202, dtype: float64

In [31]:
# Yukarıdaki eşik değeri biz 20 olarak belirledik, lakin bunun için daha sistematik bir yaklaşım geliştirmemiz gerekirse;

# Kullanıcının izlediği filmlerin %60'ını izleyen kişiler gibi bir sınır belirleyebiliriz.

# users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]
# percantage = len(movies_watched) * 60 / 100

### Adım 4: Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi

In [32]:
# Bunun için 3 adım gerçekleştireceğiz:

# 1. "28941" kullanıcı numaralı kişi ve diğer kullanıcıların verilerini bir araya getireceğiz.
# 2. Korelasyon df'ini oluşturacağız.
# 3. En benzer bullanıcıları (Top Users) bulacağız

In [33]:
movies_watched_df[movies_watched_df.index.isin(users_same_movies)]

# Yukarıda "movies_watched_df" üzerinde "users_same_movies" listesinin elemanlarını getirttik.

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),...,Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,
130.0,4.0,3.0,NaN,3.0,3.0,NaN,NaN,3.0,5.0,5.0,...,NaN,3.0,NaN,5.0,NaN,NaN,3.0,5.0,NaN,3.0
156.0,3.0,NaN,NaN,5.0,5.0,3.0,NaN,NaN,4.0,5.0,...,NaN,NaN,4.0,5.0,NaN,4.0,4.0,5.0,3.0,4.0
158.0,2.0,1.0,4.0,4.0,3.0,5.0,NaN,4.0,NaN,5.0,...,NaN,5.0,3.0,5.0,5.0,4.0,5.0,5.0,NaN,NaN
184.0,2.0,3.0,3.0,4.0,4.0,NaN,3.0,NaN,4.0,3.0,...,NaN,4.0,4.0,5.0,4.0,NaN,4.0,5.0,3.0,4.0
295.0,NaN,NaN,3.0,3.0,3.0,3.0,3.0,2.0,NaN,4.0,...,NaN,3.0,3.0,4.0,3.0,4.0,NaN,4.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138208.0,2.0,1.0,4.0,3.0,3.0,3.0,4.0,3.0,2.0,3.0,...,NaN,3.0,3.0,5.0,2.0,4.0,NaN,3.0,3.0,3.0
138279.0,3.0,NaN,NaN,3.0,5.0,5.0,5.0,4.0,NaN,5.0,...,NaN,5.0,NaN,5.0,NaN,4.0,NaN,5.0,3.0,3.0
138382.0,1.0,1.0,4.0,2.0,3.0,5.0,NaN,4.0,3.0,5.0,...,NaN,NaN,3.0,NaN,3.0,3.0,NaN,5.0,NaN,4.0


In [34]:
random_user_df[movies_watched]

# Burada da seçtiğimiz random_user'ın izlediği filmleri getirdik.

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),...,Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,
28941.0,3.0,2.0,2.0,3.0,3.0,5.0,4.0,5.0,3.0,3.0,...,3.0,5.0,5.0,5.0,3.0,5.0,5.0,1.0,1.0,1.0


In [35]:
# Yukarıdaki iki tabloyu birleştireceğiz.

final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)], random_user_df[movies_watched]])
final_df

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),...,Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,
130.0,4.0,3.0,NaN,3.0,3.0,NaN,NaN,3.0,5.0,5.0,...,NaN,3.0,NaN,5.0,NaN,NaN,3.0,5.0,NaN,3.0
156.0,3.0,NaN,NaN,5.0,5.0,3.0,NaN,NaN,4.0,5.0,...,NaN,NaN,4.0,5.0,NaN,4.0,4.0,5.0,3.0,4.0
158.0,2.0,1.0,4.0,4.0,3.0,5.0,NaN,4.0,NaN,5.0,...,NaN,5.0,3.0,5.0,5.0,4.0,5.0,5.0,NaN,NaN
184.0,2.0,3.0,3.0,4.0,4.0,NaN,3.0,NaN,4.0,3.0,...,NaN,4.0,4.0,5.0,4.0,NaN,4.0,5.0,3.0,4.0
295.0,NaN,NaN,3.0,3.0,3.0,3.0,3.0,2.0,NaN,4.0,...,NaN,3.0,3.0,4.0,3.0,4.0,NaN,4.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138279.0,3.0,NaN,NaN,3.0,5.0,5.0,5.0,4.0,NaN,5.0,...,NaN,5.0,NaN,5.0,NaN,4.0,NaN,5.0,3.0,3.0
138382.0,1.0,1.0,4.0,2.0,3.0,5.0,NaN,4.0,3.0,5.0,...,NaN,NaN,3.0,NaN,3.0,3.0,NaN,5.0,NaN,4.0
138415.0,1.0,NaN,3.0,4.0,3.0,5.0,NaN,NaN,NaN,4.0,...,NaN,5.0,NaN,5.0,3.0,4.0,4.0,NaN,3.0,NaN


In [36]:
# Yukarıdaki tabloya göre 3203 kullanıcı bulunmakta ve biz kullanıcıların beğenme davranışlarını (korelasyonunu) inceleyeceğimiz için,
# Sütunlarda kullanıcılar olmalıdır.

# Sütunlara kullanıcıları getirebilmek için verinin transpose'ını almamız gerekir.

final_df.T

userId,130.0,156.0,158.0,184.0,295.0,330.0,427.0,579.0,614.0,721.0,...,137949.0,137976.0,138019.0,138162.0,138208.0,138279.0,138382.0,138415.0,138483.0,28941.0
title,,,,,,,,,,,,,,,,,,,,,
Ace Ventura: Pet Detective (1994),4.0,3.0,2.0,2.0,NaN,2.0,2.5,4.0,1.0,5.0,...,2.0,1.0,2.5,3.0,2.0,3.0,1.0,1.0,1.0,3.0
Ace Ventura: When Nature Calls (1995),3.0,NaN,1.0,3.0,NaN,1.0,0.5,2.0,1.0,3.0,...,2.0,1.0,3.0,NaN,1.0,NaN,1.0,NaN,1.0,2.0
Aladdin (1992),NaN,NaN,4.0,3.0,3.0,5.0,2.0,3.0,3.5,4.0,...,NaN,2.0,3.5,4.0,4.0,NaN,4.0,3.0,3.0,2.0
"American President, The (1995)",3.0,5.0,4.0,4.0,3.0,3.0,2.0,NaN,NaN,5.0,...,NaN,3.0,3.5,5.0,3.0,3.0,2.0,4.0,3.0,3.0
Apollo 13 (1995),3.0,5.0,3.0,4.0,3.0,4.0,1.5,4.0,3.5,5.0,...,5.0,5.0,3.5,5.0,3.0,5.0,3.0,3.0,3.0,3.0
Babe (1995),NaN,3.0,5.0,NaN,3.0,4.0,3.0,4.0,2.0,NaN,...,4.0,5.0,4.5,5.0,3.0,5.0,5.0,5.0,NaN,5.0
Bullets Over Broadway (1994),NaN,NaN,NaN,3.0,3.0,3.0,NaN,NaN,NaN,3.0,...,4.0,4.0,NaN,4.0,4.0,5.0,NaN,NaN,3.0,4.0
Clueless (1995),3.0,NaN,4.0,NaN,2.0,2.0,3.0,3.0,NaN,NaN,...,5.0,4.0,4.5,4.0,3.0,4.0,4.0,NaN,3.0,5.0
Disclosure (1994),5.0,4.0,NaN,4.0,NaN,3.0,3.0,3.0,NaN,4.0,...,NaN,NaN,2.0,NaN,2.0,NaN,3.0,NaN,2.0,3.0


In [37]:
# Sütunlara kullanıcıları aldıktan sonra, korelasyonuna bakacağız.

final_df.T.corr()

userId,130.0,156.0,158.0,184.0,295.0,330.0,427.0,579.0,614.0,721.0,...,137949.0,137976.0,138019.0,138162.0,138208.0,138279.0,138382.0,138415.0,138483.0,28941.0
userId,,,,,,,,,,,,,,,,,,,,,
130.0,1.000000,0.129185,0.261034,0.148936,0.596849,0.313425,0.478142,0.481330,0.215877,3.571375e-01,...,0.136319,0.047775,-0.085554,-0.076942,0.096840,0.341224,0.214834,0.204479,0.231099,-0.051878
156.0,0.129185,1.000000,0.101749,0.685262,0.422885,0.435064,-0.371381,0.345646,0.378133,4.490887e-01,...,0.497887,0.172631,0.117985,0.306125,0.324016,0.376889,0.227689,0.017946,0.367543,-0.027226
158.0,0.261034,0.101749,1.000000,0.391555,0.371236,0.667808,0.541660,0.475673,0.686852,4.296689e-01,...,0.714715,0.547066,0.791502,0.210368,0.523762,0.601942,0.625739,0.640184,0.855485,0.427352
184.0,0.148936,0.685262,0.391555,1.000000,0.172799,0.507972,0.297653,0.128885,0.484434,3.209815e-01,...,0.439779,0.612599,0.402481,0.427504,0.291402,0.401356,0.619247,0.348311,0.412971,0.268111
295.0,0.596849,0.422885,0.371236,0.172799,1.000000,0.719195,0.178714,0.674272,0.282951,2.042179e-01,...,0.338234,-0.071474,0.161515,0.251561,0.081818,0.411474,-0.291667,-0.015462,0.773143,0.169609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138279.0,0.341224,0.376889,0.601942,0.401356,0.411474,0.501161,0.212350,0.380082,0.485532,3.258422e-01,...,0.510452,0.671103,0.414644,0.347905,0.293689,1.000000,0.351971,0.252259,0.583333,0.426883
138382.0,0.214834,0.227689,0.625739,0.619247,-0.291667,0.265927,0.106707,0.255331,0.410694,3.093377e-01,...,0.486453,0.464371,0.610134,0.149015,0.449272,0.351971,1.000000,0.415740,0.282521,-0.045278
138415.0,0.204479,0.017946,0.640184,0.348311,-0.015462,0.338401,0.309061,0.102815,0.216306,8.392497e-17,...,0.506137,0.455809,0.653091,-0.040641,0.249895,0.252259,0.415740,1.000000,0.709139,0.481604


In [38]:
# Bizim bu PİVOT TABLOSUNU sınıflandırmamız gerekiyor.

final_df.T.corr().unstack()

# Tablo değerleri, tablonun ".unstack()" metodu uygulanmadan önceki hali ile karşılaştırılabilir, özellikle ilk iki satır.
# Dolayısıyla tablomuzda duplicate değerler de olacaktır.

userId   userId  
130.0    130.0       1.000000
         156.0       0.129185
         158.0       0.261034
         184.0       0.148936
         295.0       0.596849
                       ...   
28941.0  138279.0    0.426883
         138382.0   -0.045278
         138415.0    0.481604
         138483.0    0.567603
         28941.0     1.000000
Length: 10259209, dtype: float64

In [39]:
pd.DataFrame(final_df.T.corr().unstack())

# df görünümü bu şekildedir.

0
userId  userId            
130.0   130.0     1.000000
        156.0     0.129185
        158.0     0.261034
        184.0     0.148936
        295.0     0.596849
...                    ...
28941.0 138279.0  0.426883
        138382.0 -0.045278
        138415.0  0.481604
        138483.0  0.567603
        28941.0   1.000000

[10259209 rows x 1 columns]

In [40]:
# Verimizi bir de sıralayalım;

final_df.T.corr().unstack().sort_values()

userId    userId 
28866.0   67756.0   -0.936065
67756.0   28866.0   -0.936065
60562.0   37121.0   -0.915003
37121.0   60562.0   -0.915003
34103.0   80593.0   -0.898718
                       ...   
136568.0  1376.0          NaN
136877.0  1376.0          NaN
136980.0  1376.0          NaN
137573.0  1376.0          NaN
138162.0  1376.0          NaN
Length: 10259209, dtype: float64

In [41]:
pd.DataFrame(final_df.T.corr().unstack().sort_values())

,,0
userId,userId,
28866.0,67756.0,-0.936065
67756.0,28866.0,-0.936065
60562.0,37121.0,-0.915003
37121.0,60562.0,-0.915003
34103.0,80593.0,-0.898718
...,...,...
136568.0,1376.0,NaN
136877.0,1376.0,NaN
136980.0,1376.0,NaN


In [42]:
# Verimizi sıraladıktan sonra hatırlıyoruz ki matris'den gelen bir duplicate sorunu vardı.
# Onu ise şöyle aşacağız;

final_df.T.corr().unstack().sort_values().drop_duplicates()

userId    userId 
28866.0   67756.0   -0.936065
60562.0   37121.0   -0.915003
34103.0   80593.0   -0.898718
62575.0   21398.0   -0.896612
117826.0  48416.0   -0.883969
                       ...   
96952.0   34350.0    0.971667
32344.0   68063.0    0.977387
28941.0   28941.0    1.000000
16562.0   41005.0    1.000000
725.0     43765.0         NaN
Length: 3821114, dtype: float64

In [43]:
# Şimdi bu yaptığımız tablo ile bir dataframe oluşturup sonrasında da bir değişkene atayalım;

corr_df = pd.DataFrame(final_df.T.corr().unstack().sort_values().drop_duplicates(), columns=["corr"])
corr_df

,,corr
userId,userId,
28866.0,67756.0,-0.936065
60562.0,37121.0,-0.915003
34103.0,80593.0,-0.898718
62575.0,21398.0,-0.896612
117826.0,48416.0,-0.883969
...,...,...
96952.0,34350.0,0.971667
32344.0,68063.0,0.977387
28941.0,28941.0,1.000000


In [44]:
corr_df.columns

# Aşağıya baktığımızda "corr_df"nin yalnızca tek değişkeni olduğunu görüyoruz.
# Bu da demektir ki ilk iki sütun gibi gözüken yerler indeks'tir.
# Yani bir dataframe'in birden fazla indeksi OLABİLİRMİŞ.

# Drop duplicate yaptığımızda satır sayısı "10.259.209" dan "3.821.114" a düşmüştür.
# Yani "NaN" değerler de dataframeimizden drop edilmiştir.

Index(['corr'], dtype='object')

In [45]:
# Şimdi indek değerlerimizi isimlendirelim;

corr_df.index.names = ["user_id_1", "user_id_2"]

In [46]:
# corr_df'i indekslerinden kurtarmamız gerekecek;

corr_df = corr_df.reset_index()

In [47]:
corr_df

,user_id_1,user_id_2,corr
0,28866.0,67756.0,-0.936065
1,60562.0,37121.0,-0.915003
2,34103.0,80593.0,-0.898718
3,62575.0,21398.0,-0.896612
4,117826.0,48416.0,-0.883969
...,...,...,...
3821109,96952.0,34350.0,0.971667
3821110,32344.0,68063.0,0.977387
3821111,28941.0,28941.0,1.000000
3821112,16562.0,41005.0,1.000000


In [48]:
# Bizim artık "28941" kullanıcı numaralı kişi ile yüksek korelasyonlu kişileri bulmamız gerekir;

# Bu sebeple user_id_1'e "28941" kişiyi almamız,
# user_id_2'yi de büyükten küçüğe doğru sıralamamız gerekmektedir. 

corr_df[corr_df["user_id_1"] == random_user]

,user_id_1,user_id_2,corr
9319,28941.0,103594.0,-0.568855
20036,28941.0,61951.0,-0.505154
24931,28941.0,7916.0,-0.485621
26633,28941.0,89103.0,-0.479758
31217,28941.0,26929.0,-0.465009
...,...,...,...
3777415,28941.0,9783.0,0.747942
3788717,28941.0,127259.0,0.763925
3802642,28941.0,101628.0,0.790405
3806621,28941.0,45158.0,0.800749


In [49]:
# korelasyonu %65'ten yüksek olan kişileri de seçelim.

corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)]

,user_id_1,user_id_2,corr
3665525,28941.0,132307.0,0.660726
3666440,28941.0,42696.0,0.661268
3673899,28941.0,1362.0,0.665461
3681005,28941.0,124546.0,0.669585
3681942,28941.0,73844.0,0.670105
3688267,28941.0,66603.0,0.673862
3698231,28941.0,58705.0,0.680166
3700221,28941.0,131285.0,0.681491
3700698,28941.0,125901.0,0.681795
3721984,28941.0,91954.0,0.696581


In [50]:
# Sadece "user_id_2" ve "corr" sütunlarını istersek;

corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]]

,user_id_2,corr
3665525,132307.0,0.660726
3666440,42696.0,0.661268
3673899,1362.0,0.665461
3681005,124546.0,0.669585
3681942,73844.0,0.670105
3688267,66603.0,0.673862
3698231,58705.0,0.680166
3700221,131285.0,0.681491
3700698,125901.0,0.681795
3721984,91954.0,0.696581


In [51]:
# indeksleri de resetleyelim ve bir değişkene atayalım;
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]].reset_index(drop=True)
top_users

,user_id_2,corr
0,132307.0,0.660726
1,42696.0,0.661268
2,1362.0,0.665461
3,124546.0,0.669585
4,73844.0,0.670105
5,66603.0,0.673862
6,58705.0,0.680166
7,131285.0,0.681491
8,125901.0,0.681795
9,91954.0,0.696581


In [52]:
# top_users dataframe'mimizi sıralayalım;

top_users = top_users.sort_values(by="corr", ascending=False)
top_users

,user_id_2,corr
27,28941.0,1.000000
26,45158.0,0.800749
25,101628.0,0.790405
24,127259.0,0.763925
23,9783.0,0.747942
22,93089.0,0.738536
21,136259.0,0.737177
20,136259.0,0.737177
19,67346.0,0.732943
18,67346.0,0.732943


In [53]:
# dataframe'mimizin sütun ismini de değiştirelim zira artık sadece bir userId sütununa ihtiyacımız var.

top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
top_users

,userId,corr
27,28941.0,1.000000
26,45158.0,0.800749
25,101628.0,0.790405
24,127259.0,0.763925
23,9783.0,0.747942
22,93089.0,0.738536
21,136259.0,0.737177
20,136259.0,0.737177
19,67346.0,0.732943
18,67346.0,0.732943


In [54]:
# Yukarıda "28941" kullanıcı numaralı kişi ile en yüksek korelasyona sahip kişileri bulduk;
# Lakin bize bu kullanıcıların hangi filme kaç puan verdiği bilgisi gerekmektedir.

# Bunun için bir df çağıralım;

rating

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [55]:
# Bize sadece "userId", "movieId" ve "rating" sütunları gerekmektedir.

rating[["userId", "movieId", "rating"]]

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5
...,...,...,...
20000258,138493,68954,4.5
20000259,138493,69526,4.5
20000260,138493,69644,3.0
20000261,138493,70286,5.0


In [56]:
# Yani bizim "rating" dataframe'i ile top users dataframe'ini birleştirmemiz gerekir;

top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how="inner")

In [57]:
top_users_ratings

,userId,corr,movieId,rating
0,28941.0,1.000000,7,5.0
1,28941.0,1.000000,11,3.0
2,28941.0,1.000000,17,5.0
3,28941.0,1.000000,19,2.0
4,28941.0,1.000000,34,5.0
...,...,...,...,...
13680,132307.0,0.660726,593,4.0
13681,132307.0,0.660726,595,4.0
13682,132307.0,0.660726,597,4.0
13683,132307.0,0.660726,608,5.0


In [58]:
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]
top_users_ratings

,userId,corr,movieId,rating
33,45158.0,0.800749,1,1.5
34,45158.0,0.800749,3,1.0
35,45158.0,0.800749,17,3.5
36,45158.0,0.800749,19,2.0
37,45158.0,0.800749,22,3.0
...,...,...,...,...
13680,132307.0,0.660726,593,4.0
13681,132307.0,0.660726,595,4.0
13682,132307.0,0.660726,597,4.0
13683,132307.0,0.660726,608,5.0


### Adım 5: Weighted Average Recommendation Score'un Hesaplanması

In [59]:
# Yukarıdaki "top_users_ratings tablosunda kişilerin;
# "28941" kullanıcı numaralı kişi ile korelasyonları,
# kişilerin izledikleri filmler,
# ve kişilerin izledikleri filmlere verdikleri puanlar bulunmaktadır.

# Biz izlenilen filmleri "groupby"a alıp yüksek puana göre yukarıdan aşağa sıralarsak, 
# "28941" kullanıcı numaralı kişi'ye önerilecek filmleri bulabiliriz.

# Lakin Bu yöntem ile korelasyonu göz ardı etmiş oluruz.
# Dolayısıyla bizim ağırlıklı ortalama değikeni oluşturup, 
# hem bu filmlerin ortalamasını yansıtacak hem de korelasyonu yansıtacak bir hesaplama yapmamız gerekir.

top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']

/var/folders/l2/g3cwllyj4_139gx51n27kyn80000gn/T/ipykernel_39901/3363779502.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']


In [60]:
top_users_ratings

,userId,corr,movieId,rating,weighted_rating
33,45158.0,0.800749,1,1.5,1.201124
34,45158.0,0.800749,3,1.0,0.800749
35,45158.0,0.800749,17,3.5,2.802622
36,45158.0,0.800749,19,2.0,1.601498
37,45158.0,0.800749,22,3.0,2.402248
...,...,...,...,...,...
13680,132307.0,0.660726,593,4.0,2.642904
13681,132307.0,0.660726,595,4.0,2.642904
13682,132307.0,0.660726,597,4.0,2.642904
13683,132307.0,0.660726,608,5.0,3.303630


In [61]:
# Şimdi filmlere göre "weighted_rating" ortalamalarını groupby'a alırsak;

recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

In [62]:
recommendation_df

,weighted_rating
movieId,
1,2.205824
2,1.463950
3,1.181772
4,1.642769
5,1.151408
...,...
56367,2.565300
56508,2.931772
56788,2.565300


In [63]:
# Indekslerimizi sütunlara almak için;

recommendation_df = recommendation_df.reset_index()

In [64]:
# 3.5 puandan fazla olan filmlerin önerisini yapacağımızı düşünecek olursak;

recommendation_df[recommendation_df["weighted_rating"] > 3.5]

,movieId,weighted_rating
26,30,3.952023
45,53,3.952023
183,242,3.952023
201,263,3.649974
257,326,3.952023
394,501,3.679739
799,1201,3.642581
848,1257,3.603372
920,1348,3.692678
1543,2496,3.642581


In [65]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False)
movies_to_be_recommend

,movieId,weighted_rating
26,30,3.952023
257,326,3.952023
183,242,3.952023
45,53,3.952023
920,1348,3.692678
394,501,3.679739
3873,25850,3.664714
3691,8128,3.664714
3907,26394,3.664714
3622,7585,3.664714


In [66]:
# Şimdi "movie" df'si ile "movies_to_be_recommend" df'sini birleştireceğiz.

movies_to_be_recommend.merge(movie[["movieId", "title"]])


,movieId,weighted_rating,title
0,30,3.952023,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
1,326,3.952023,To Live (Huozhe) (1994)
2,242,3.952023,Farinelli: il castrato (1994)
3,53,3.952023,Lamerica (1994)
4,1348,3.692678,"Nosferatu (Nosferatu, eine Symphonie des Graue..."
5,501,3.679739,Naked (1993)
6,25850,3.664714,Holiday (1938)
7,8128,3.664714,Au revoir les enfants (1987)
8,26394,3.664714,"Turning Point, The (1977)"
9,7585,3.664714,Summertime (1955)
